In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from time import strptime
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv("C:/Users/ZHANG FANYUE/Desktop/NTU/Data Mining/Project/bank-full.csv", sep = ";")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [4]:
#change month to numeric
data['month'] = [strptime(str(x), '%b').tm_mon for x in data['month']]

#change all object data type to categorical
list_str_obj_cols = data.columns[data.dtypes == "object"].tolist()
for str_obj_col in list_str_obj_cols:
    data[str_obj_col] = data[str_obj_col].astype("category")

#encode all categorical data
df_encoded = pd.get_dummies(data, columns=['job', 'marital', 'education', 'default', 
                                           'housing', 'loan', 'contact', 'poutcome'],)


#standardize all numeric data
data_numeric = data[["age", "balance", "day", "duration", "campaign", "pdays", "previous", "month"]]
 
std_scaler = StandardScaler()
df_scaled = std_scaler.fit_transform(data_numeric.to_numpy())
df_scaled = pd.DataFrame(df_scaled, columns = ["age", "balance", "day", "duration", "campaign", "pdays", "previous", "month"])
 
#combine both datasets
df_encoded.update(df_scaled)

#change class label to 0 and 1
df_encoded.y = pd.Categorical(df_encoded.y).codes
newdata = df_encoded

In [5]:
pd.set_option('display.max_columns', None)
newdata

,age,balance,day,month,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,1.606965,0.256419,-1.298476,-0.475354,0.011016,-0.569351,-0.411453,-0.251940,0,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,True,True,False,False,False,True,False,False,False,True
1,0.288529,-0.437895,-1.298476,-0.475354,-0.416127,-0.569351,-0.411453,-0.251940,0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,False,False,True,True,False,False,False,True,False,False,False,True
2,-0.747384,-0.446762,-1.298476,-0.475354,-0.707361,-0.569351,-0.411453,-0.251940,0,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,True,False,False,True,False,False,False,True
3,0.571051,0.047205,-1.298476,-0.475354,-0.645231,-0.569351,-0.411453,-0.251940,0,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,True,True,False,False,False,True,False,False,False,True
4,-0.747384,-0.447091,-1.298476,-0.475354,-0.233620,-0.569351,-0.411453,-0.251940,0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,True,False,True,False,True,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,0.947747,-0.176460,0.143418,2.016333,2.791329,0.076230,-0.411453,-0.251940,1,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,True,False,True,False,True,False,True,False,False,False,False,False,True
45207,2.831227,0.120447,0.143418,2.016333,0.768224,-0.246560,-0.411453,-0.251940,1,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,True,False,True,False,True,False,False,False,False,False,True
45208,2.925401,1.429593,0.143418,2.016333,3.373797,0.721811,1.436189,1.050473,1,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,True,False,True,False,True,False,False,False,False,True,False
45209,1.512791,-0.228024,0.143418,2.016333,0.970146,0.399020,-0.411453,-0.251940,0,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,True,False,True,False,False,True,False,False,False,False,True


In [6]:
X = newdata.drop(["y"], axis=1)
y = newdata["y"]

# Basic

In [7]:
from sklearn.model_selection import cross_validate 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, matthews_corrcoef


#mcc_scorer = make_scorer(matthews_corrcoef)
model = MLPClassifier() 
cv_results = cross_validate(model, X, y, cv=5, 
                            return_train_score=True, 
                            scoring= {"MCC": make_scorer(matthews_corrcoef), "Accuracy": "accuracy"}) 
cv_results

C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ZHANG FANYUE\anaconda3\Lib\site-pac

{'fit_time': array([39.75194669, 40.08258176, 40.28624725, 41.24296498, 42.06954074]),
 'score_time': array([0.0150857 , 0.01521826, 0.01189208, 0.01546001, 0.02760506]),
 'test_MCC': array([ 0.10817797, -0.0078684 ,  0.00468848, -0.05339727,  0.32783922]),
 'train_MCC': array([0.66183437, 0.70592128, 0.69916156, 0.73768746, 0.72126897]),
 'test_Accuracy': array([0.88455159, 0.79882769, 0.78522451, 0.58493696, 0.70957753]),
 'train_Accuracy': array([0.93394714, 0.94191158, 0.94218806, 0.9478559 , 0.94730294])}

In [8]:
import numpy as np
print("train MCC:",np.mean(cv_results['train_MCC']))
print("test MCC:",np.mean(cv_results['test_MCC']))

train MCC: 0.705174729099817
test MCC: 0.07588800069440707


In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef, accuracy_score
import numpy as np
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE

# Initialize the nn model
model = MLPClassifier() 

# Initialize Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# For storing results
train_mcc_scores = []
test_mcc_scores = []

# Loop through each fold
for train_index, test_index in skf.split(X, y):
    # Split the data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    
    # Apply SMOTE only to the training set
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    
    
    # Train the model on the resampled training data
    model.fit(X_train_resampled, y_train_resampled)
    
    # Predict and compute MCC for the training data
    train_preds = model.predict(X_train_resampled)
    train_mcc = matthews_corrcoef(y_train_resampled, train_preds)
    train_mcc_scores.append(train_mcc)
    
    # Predict and compute MCC for the testing data
    test_preds = model.predict(X_test)
    test_mcc = matthews_corrcoef(y_test, test_preds)
    test_mcc_scores.append(test_mcc)

# Print the average MCC scores
print("Train MCC:", np.mean(train_mcc_scores))
print("Test MCC:", np.mean(test_mcc_scores))

C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Train MCC: 0.8922334745670332
Test MCC: 0.47698554539196464


# Hyperparameter Tuning

In [10]:
#used for parameter tuning
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)

In [11]:
#smote on training dataset
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, matthews_corrcoef

model = MLPClassifier() 
parameters={
'hidden_layer_sizes': [(100), (50,50), (20,20,20)],
'alpha': [0.001, 0.0001, 0.01],
'activation': ["logistic", "relu", "Tanh"],
'learning_rate_init': [0.001, 0.0001, 0.00001],
'max_iter':[200, 500]
}

tuning = GridSearchCV(estimator=model,param_grid=parameters,n_jobs=-1,verbose=2,cv=5, refit=True,
                      scoring = make_scorer(matthews_corrcoef))
tuning.fit(X_resampled, y_resampled)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
270 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
62 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "C:\Users\ZHANG FANYUE\anaconda3\Lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\ZHAN

GridSearchCV(cv=5, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['logistic', 'relu', 'Tanh'],
                         'alpha': [0.001, 0.0001, 0.01],
                         'hidden_layer_sizes': [100, (50, 50), (20, 20, 20)],
                         'learning_rate_init': [0.001, 0.0001, 1e-05],
                         'max_iter': [200, 500]},
             scoring=make_scorer(matthews_corrcoef), verbose=2)

In [13]:
pred = tuning.predict(X_test) #predict using test data
matthews_corrcoef(y_test, pred)

0.4405717288120462

In [14]:
print("Best Parameters: ", tuning.best_params_)
print("Best Score: ", tuning.best_score_)

Best Parameters:  {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001, 'max_iter': 500}
Best Score:  0.8509925933085917


In [15]:
tuning.cv_results_

{'mean_fit_time': array([1.21855096e+02, 3.16475309e+02, 1.24259650e+02, 3.12925141e+02,
        1.31781701e+02, 3.20252765e+02, 1.58460001e+02, 2.96449528e+02,
        1.58207903e+02, 3.56672037e+02, 1.55544811e+02, 3.85098110e+02,
        1.14883454e+02, 1.28828140e+02, 1.19807325e+02, 2.57156225e+02,
        9.79276225e+01, 3.08389128e+02, 1.27901444e+02, 3.16182273e+02,
        1.30404039e+02, 3.15640165e+02, 1.29483182e+02, 3.24637975e+02,
        1.53911017e+02, 3.67038201e+02, 1.57971160e+02, 3.23591070e+02,
        1.57089520e+02, 3.82026189e+02, 1.21486761e+02, 1.26033466e+02,
        1.23149637e+02, 2.29668730e+02, 1.20098554e+02, 2.42522604e+02,
        1.28164568e+02, 2.28871696e+02, 1.29773295e+02, 2.87685947e+02,
        1.28061751e+02, 3.14874921e+02, 1.59332596e+02, 1.85271664e+02,
        1.55851072e+02, 3.19084465e+02, 1.57149929e+02, 3.91641411e+02,
        1.07267647e+02, 1.04787853e+02, 1.19256292e+02, 2.18340974e+02,
        9.62599054e+01, 2.39133643e+02, 1.38618

In [16]:
# Initialize the nn model
model = MLPClassifier(activation='relu', alpha= 0.0001, 
                      hidden_layer_sizes= (50, 50), learning_rate_init= 0.001, max_iter= 500) 

# Initialize Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# For storing results
train_mcc_scores = []
test_mcc_scores = []
train_accuracy_scores = []
test_accuracy_scores = []

# Loop through each fold
for train_index, test_index in skf.split(X, y):
    # Split the data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    
    # Apply SMOTE only to the training set
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    
    
    # Train the model on the resampled training data
    model.fit(X_train_resampled, y_train_resampled)
    
    # Predict and compute MCC for the training data
    train_preds = model.predict(X_train_resampled)
    train_mcc = matthews_corrcoef(y_train_resampled, train_preds)
    train_mcc_scores.append(train_mcc)
    
    # Predict and compute MCC for the testing data
    test_preds = model.predict(X_test)
    test_mcc = matthews_corrcoef(y_test, test_preds)
    test_mcc_scores.append(test_mcc)
    accuracy_1 = accuracy_score(y_train_resampled, train_preds)
    accuracy_2 = accuracy_score(y_test, test_preds)
    train_accuracy_scores.append(accuracy_1)
    test_accuracy_scores.append(accuracy_2)
    print("Train Accuracy:", accuracy_1)
    print("Test Accuracy:", accuracy_2)

# Print the average MCC scores
print("Train MCC:", np.mean(train_mcc_scores))
print("Test MCC:", np.mean(test_mcc_scores))
print("Train Accuracy_mean:", np.mean(accuracy_1))
print("Test Accuracy_mean:", np.mean(accuracy_2))

print("Average Train Accuracy:", np.mean(train_accuracy_scores))
print("Average Test Accuracy:", np.mean(test_accuracy_scores))

Train Accuracy: 0.9545198359269812
Test Accuracy: 0.8795753621585757
Train Accuracy: 0.9570247675110374
Test Accuracy: 0.8754700287547003
Train Accuracy: 0.9630221053290751
Test Accuracy: 0.8692767086927671
Train Accuracy: 0.9528617947272842
Test Accuracy: 0.8941605839416058
Train Accuracy: 0.9598597282234329
Test Accuracy: 0.8730369387303694
Train MCC: 0.9153576639780662
Test MCC: 0.4531551463959021
Train Accuracy_mean: 0.9598597282234329
Test Accuracy_mean: 0.8730369387303694
Average Train Accuracy: 0.9574576463435622
Average Test Accuracy: 0.8783039244556037


In [17]:

print("Train Accuracy:", np.mean(accuracy_1))
print("Test Accuracy:",np.mean(accuracy_2))

Train Accuracy: 0.9598597282234329
Test Accuracy: 0.8730369387303694
